In [23]:
import requests
from io import BytesIO
import rasterio
import xml.etree.ElementTree as ET

In [25]:


state = "uttar_pradesh"
district = "jaunpur"
block = "badlapur"

GEOSERVER_WCS_URL = "https://geoserver.core-stack.org:8443/geoserver/LULC_level_1/wcs"
coverage_id = "LULC_level_1:LULC_23_24_badlapur_level_1"

bbox = "82.2,25.45,82.35,25.60"


In [26]:
params = {
    "service": "WCS",
    "version": "2.0.1",
    "request": "GetCoverage",
    "CoverageId": coverage_id,
    "bbox": bbox,
    "format": "geotiff"
}

try:

    response = requests.get(GEOSERVER_WCS_URL, params=params, verify=False)
    response.raise_for_status()

    raster_data = BytesIO(response.content)

    with rasterio.open(raster_data) as src:
        gsd_x, gsd_y = src.res
        print(f"Ground Sample Distance (GSD):")
        print(f"Pixel Width (x): {gsd_x}")
        print(f"Pixel Height (y): {gsd_y}")
        
        width, height = src.shape
        print(f"Image Dimensions:")
        print(f"  Width: {width} pixels")
        print(f"  Height: {height} pixels")

        if src.crs:
            epsg_code = src.crs.to_string()
            print(f"  CRS: {epsg_code}")
        else:
            print("EPSG Code: Not found or not valid.")
            
except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching data from GeoServer: {e}")

/home/vishnu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoserver.core-stack.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Ground Sample Distance (GSD):
Pixel Width (x): 8.983152841195215e-05
Pixel Height (y): 8.983152841195215e-05
Image Dimensions:
  Width: 3150 pixels
  Height: 5442 pixels
  CRS: EPSG:4326


### Above part was with bbox hardcorded , now will try to get bounding box from geoserver metadata and use that for getting the GSD, image dimensin etc

In [ ]:
from owslib.wcs import WebCoverageService


wcs_url = "https://geoserver.core-stack.org:8443/geoserver/LULC_level_1/wcs"
coverage_id = "LULC_23_24_badlapur_level_1"

try:
    
    wcs = WebCoverageService(wcs_url, version='2.0.1')

    
    coverage = wcs[coverage_id]

    bbox = coverage.boundingBox
    print(f"Bounding box: {bbox}")

    grid = coverage.grid
    width = grid['width']
    height = grid['height']
    print(f"Raster dimensions (Width x Height): {width} x {height}")

    gsd_x = (bbox[2] - bbox[0]) / width
    gsd_y = (bbox[3] - bbox[1]) / height
    print(f"Ground Sample Distance (GSD): {gsd_x:.5f} units/pixel (X), {gsd_y:.5f} units/pixel (Y)")

except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 'No content named LULC_23_24_badlapur_level_1'
